- 트랜스퍼 모델 : 2017
  - 병렬처리
  - 인코더, 디코더, 셀프 어텐션 아키텍처를 사용
  - 셀프어텐션
    - 입력데이터에 대한 쿼리(query), 키(key), 값(value) --> 스코어계산 -> 소프트 맥스 함수를 통해 정규화 각 단어의 가중치를 얻고, 이를사용해서 최종 출력
  - 인코더 : 컨텍스트 (입력데이터를 처리해서 의미있는 표현), 여러층의 셀프어텐 , 뉴럴네트웍크로 성
  - 디코더 : 인코더의 출력과 목표데이터를 입력으로 받아서 최종 출력

  - seq2seq VS 트랜스포머
    - RNN/GRU/LSTM 기반 VS 어텐션기반
      - RNN 계열을이용해서 순차적용 / 어텐션매커니즘을 활용 순차처리없이 병렬처리, 긴 시퀀스 처리에 효율적
    - 어텐션매커니즘
      - seq2seq는 인코더와 디코더 사이에 어텐션을 추가한
      - 트랜스포머 셀프어텐션을 사용해서 각 단어가 시퀀스 내의 다른 모든 단어와의 연관성을 스스로 이해
      - 쿼리,키,벨류 벡터
      - 문장 : 고양이가 방에서 뛰어다닌다
      - 쿼리 : 고양이
      - 키:  방에서       
             뛰어다닌다
      - 벡터 내적값을구한다  -- 유사도 기반
      - 스코어 합이 1이 되도록 정규화-가중치(확률분포형태)

    - 어텐션이 문장을 이해하는 과정
      - 문장 : 고양이가 쥐를 쫓는다
      - 고양이, 쥐, 쫓는다 각각의 쿼리,키, 값이 생
      - 어텐션 스코어 : 고양이 쿼리벡터 쥐, 쫓는다 키 벡터를 비교 스코어를 계산 (ex 고양이하고 쫓는다 스코어가 높게)
      - 정규화 : softmax
      - 최종출력 : 고양이라는 입력에 대해서 출력은 관련성에따라서 쫒는다 벡터가 더큰 비중으로 가중합


  - 디코더
    - 순차적으로 처리 / 병렬로처리
    - 단일어텐션 매커니즘 / 멀티해드 어텐션(전체문맥을 고려한 어텐션)
    - 이전상태에 의존을크게 / 시간관련 정보도 포함
  
  - 셀프어텐션
    - 입력시퀀스의 모든  토큰이 각각의 모든 토큰과 관계를 자체적으로 비교해서 각 토큰의 중요성을 할당
  - 멀티해드 어텐션
    - 헤드를 사용: 고유한 가중치 메트릭스를 사용,쿼리,키,값벡터포함
    - 헤드사이의 어텐션 스코어계산






In [1]:
!pip install tensorflow==2.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 51.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

In [2]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [3]:
import gdown
file_id = '1cjc0l-hwP2JTKDqYVKmIZ4ppFuA4ooVo'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'data.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1cjc0l-hwP2JTKDqYVKmIZ4ppFuA4ooVo
To: /content/data.zip
100%|██████████| 4.03k/4.03k [00:00<00:00, 12.2MB/s]


'data.zip'

In [4]:
!unzip data.zip

Archive:  data.zip
  inflating: train_outputs.npy       
  inflating: train_inputs.npy        
  inflating: data_configs.json       
  inflating: vocabulary.txt          
  inflating: train_targets.npy       


In [5]:
# 파일 로드
import numpy as np
import json

index_inputs = np.load(open('train_inputs.npy', 'rb'))
index_outputs = np.load(open('train_outputs.npy', 'rb'))
index_targets = np.load(open('train_targets.npy', 'rb'))
prepro_configs = json.load(open('data_configs.json', 'r'))

In [6]:
char2idx = prepro_configs['char2idx']
end_index = prepro_configs['end_symbol']
model_name = 'transformer'
vocab_size = prepro_configs['vocab_size']
BATCH_SIZE = 2
MAX_SEQUENCE = 25
EPOCHS = 30
VALID_SPLIT = 0.1

kargs = {'model_name': model_name,
         'num_layers': 2,
         'd_model': 512,
         'num_heads': 8,
         'dff': 2048,
         'input_vocab_size': vocab_size,
         'target_vocab_size': vocab_size,
         'maximum_position_encoding': MAX_SEQUENCE,
         'end_token_idx': char2idx[end_index],
         'rate': 0.1
        }

In [7]:
# 포워드 마스킹  :시퀀스데이터 대해 패딩 마스크생성-> 패딩된 정보를 무시하고 중요한 정보에 주의를 기울
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32) # trhe, false -> 0.0 ,1.0
    return seq[:,tf.newaxis, tf.newaxis,:]  # (batch_size,seq_len) -> (batch_size, 1,1,seq_len)
    # Scaled dot-product Attention  스케일드 점수기반 어텐

In [8]:
# lower triangular Matrix  하삼각 행렬 : 대각선(좌상->우하)아래의 값은 0가 아니고 위쪽은 전부 0인
# 임계치보다 크거나 같으면 아래쪽... .나머지는 모두 0
# 트랜스포머 look ahead mask(미래를 내다보는 마스크) : 현재와 과거의 정보만 사용하도록
# 각 토큰이 자신보다 이후에 있는 토큰을 참조하지 않도록 제한
# 1 0 0 0 첫번재 토큰(자신만 본다)
# 1 1 0 0 두번재 토큰은 첫번째와 두번재 토큰을 본다.
# 1 1 1 0 세번째 토큰은 첫번재와 두번재 세번째 토큰을
# 1 1 1 1


In [9]:
# 첫번재 토큰은 현재를 제외한 나머지를 마스킹
1- tf.linalg.band_part(tf.ones((5, 5)), -1, 0) # # 현재와 이전 토큰만 보도록 설정

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

In [10]:
# 어해드 마스크
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask
# 패딩과 미래의 토큰 참조를 차단
def create_masks(inp,tar):
  '''
  inp 인코더에 전달될 입력 시퀀스
  tar 디코더에 전달될 타겟 시퀀스
  '''
  enc_padding_mask = create_padding_mask(inp) # 패딩토큰( 0 )을 마스킹
  dec_padding_mask = create_padding_mask(inp) # 패딩토큰( 0 )을 마스킹
  dec_target_padding_mask = create_padding_mask(tar) # 패딩토큰( 0 )을 마스킹

  # 타겟의 길이만큼 하삼각형 행렬
  # 첫번째 어텐션 블럭에서 미래의 토큰 참조 방지
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1]) # 미래
  # 마스크 결합 : 두개중에 더 큰 값을 취함
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask) # 패딩과 미래
  return enc_padding_mask, combined_mask, dec_padding_mask


In [11]:
enc_padding_mask, look_ahead_mask, dec_padding_maks =  create_masks(index_inputs, index_outputs)

In [12]:
# 포지셔널 인코딩 : 트랜스포머 모델에서 입력시퀀스의 위치 정보를 임베딩에 추가
# RNN 계열을 사용하지 않아서 순환구조가 없기때문에 순서를 인식시킬수 있도록 위치정보가필요
# 위치 정보를추가해서 각 단어의 상대적인 위치를 알 수 있게
# 각도 비율 계산 : 각 차원에서 사용하는 주파수 값을 계산
def get_angles(pos, i, d_model):
  '''
  pos : 시퀀스에서의 위치
  i : 포지션의 차원 인덱스
  d_model : 모델의 차원
  '''
  # i//2 짝 홀을 구분 -> 사인함수 코사인함수를 사용하기 위한 패턴
  angle_rates = 1 / np.power(10000, (2 * i//2) / np.float32(d_model)) # 주파수 기반의 스케일링
  return pos * angle_rates # 위치 * 주파수비율


def positional_encoding(position, d_model):
  '''
  position : 시퀀스의 길이
  d_model : 임베딩 벡터의 차원
  '''
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                        np.arange(d_model)[np.newaxis, :],
                        d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  # 사인과 코사인을 주기적으로 사용는 이유 --> 성능향상

  pos_encoding = angle_rads[np.newaxis, ...] # shape 모양을 맞춰줌

  return tf.cast(pos_encoding, dtype=tf.float32) # 텐서로 변환

In [13]:
# 시퀀스에서 어떤 부분에 더 주의를 기울일지 결정
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """
    # 넘파이 a.dot(b)  a@b   닷 프로덕트-->내적연산
    # 각 쿼리가 모든 키와 얼마나 유사한지 측정, 어텐션 스코어
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    # v(값) * 어텐션 가중치의 행렬곱 => 최종 어텐션 생성
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [14]:
# 트랜스포머 모델


# 패딩(빈자리 매꾸기 0), 마스킹(의미없는 정보를 참조하지 않도록 참조하지 않는 위치를 얻기위해서 해당 값이 1)

# 포지셔널 인코딩(순차처리를 위해서 위치정보. 인덱스 짝수면 싸인함수 홀수면 코사인함수 )

# 어텐션 : 토큰에 가중치를 부여(중요한 단어에)
  # 참고 : 셀프 어텐션 : 각 단어가 자신을 포함한 모든 단어와 관련있는지 학습해서 중요 정보 찾는게

# 멀티 헤드 어텐션 : 하나의 어텐션만 사용할 경우 모델이 특정한 부분에 과도하게 집중을 방지
  # 여러개의 어텐션 메커니즘을 병렬로 적용(어텐션 헤드를 사용해서 서로 다른 정보의 관계를 학습)
  # 각 헤드가 독립적으로 다른 부분에 집중 :
    # 하나의 헤드가 특정 단어간의 관계를 집중적으로 학습하고 있을때
    # 다른 헤드는 또 다른 단어간의 관계를 학습
    # ex ) 한 헤드 주어와 동사간의 관계, 또 다른 헤드는 형용사와 명사간의 관계 학습
    # ex) The cat sat on the mat
    # 첫번째 헤더 : The, cat
    # 두번째 헤더 : cat, on
    # 세번째 헤더 : 전체문장구조

# 피드 포워드 네트웍(Feed Forwad Network)
  # 어텐션 이후에 정보를 처리하는 레이어(완전연결레이어.) -- FC 학습.. 비선형구조로

# 인코더 레이어: 어텐션 + 피드포워드 네트웍으로 구성

# 디코더 레이어 : 인코더 레이어와 비슷  두가지 어텐션메커니즘사용
  # 셀프어텐션 : 입력 시퀀스간의 관계를 학습
  # 인코더 출력과 디코더의 입력간의 관계를 학습-> 마스킹을 이용해서 미래시점의 정보 참조 방지

# 인코더 : 인코더 레이어 여러개를 사용(연결)
# 디코더 : 출력 예측 , 디코더 레이어 연결

# 트랜스포머 모델 : 인코더와 디코더를 연결

# 로스함수, accuracy함수 : 교차엔트로피 로스 함수

# 학습 : 컴파일, 콜벡.. 학습
# 평가 : 평가
# 예측 : 답변

In [15]:
def test(x,y):
  assert y > 0
  print(x / y)

test(5,0)

AssertionError: 

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = kargs['num_heads']  # 헤더수
        self.d_model = kargs['d_model'] # 전체 입력벡터의 크기

        # 일관성유지를 위해 동일한 차원수를 가지도록
        assert self.d_model % self.num_heads == 0
        # 각 헤드에 배분할수 있는 차원 수
        self.depth = self.d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(kargs['d_model'])
        self.wk = tf.keras.layers.Dense(kargs['d_model'])
        self.wv = tf.keras.layers.Dense(kargs['d_model'])

        self.dense = tf.keras.layers.Dense(kargs['d_model'])  # 모든 헤더의 출력을 결합

    # 입력텐서를 여러개의 헤드로 나눔 각 헤드가 독립적으로 계산-> 핵심
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3]) # batch_size, num_head, seq_len, depth

    def __call__(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask) # 입력의 중요한 부분에 집중

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

def point_wise_feed_forward_network(**kargs):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(kargs['dff'], activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(kargs['d_model'])  # (batch_size, seq_len, d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(**kargs)
        self.ffn = point_wise_feed_forward_network(**kargs)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(kargs['rate'])
        self.dropout2 = tf.keras.layers.Dropout(kargs['rate'])

    def call(self, x, mask):
        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(**kargs) # 셀프어텐션을 위함
        self.mha2 = MultiHeadAttention(**kargs) # 인코더-디코더용

        self.ffn = point_wise_feed_forward_network(**kargs) # 어텐션 결과에 비선형을 적용 표현력을 높임

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(kargs['rate'])
        self.dropout2 = tf.keras.layers.Dropout(kargs['rate'])
        self.dropout3 = tf.keras.layers.Dropout(kargs['rate'])


    def __call__(self, x, enc_output, look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3, attn_weights_block1, attn_weights_block2

class Encoder(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(Encoder, self).__init__()

        self.d_model = kargs['d_model']
        self.num_layers = kargs['num_layers']

        self.embedding = tf.keras.layers.Embedding(kargs['input_vocab_size'], self.d_model)
        self.pos_encoding = positional_encoding(kargs['maximum_position_encoding'],
                                                self.d_model)


        self.enc_layers = [EncoderLayer(**kargs)
                           for _ in range(self.num_layers)]

        self.dropout = tf.keras.layers.Dropout(kargs['rate'])

    def call(self, x, mask):
        seq_len = tf.shape(x)[1]
        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, mask)

        return x  # (batch_size, input_seq_len, d_model)
class Decoder(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(Decoder, self).__init__()

        self.d_model = kargs['d_model']
        self.num_layers = kargs['num_layers']

        self.embedding = tf.keras.layers.Embedding(kargs['target_vocab_size'], self.d_model)
        self.pos_encoding = positional_encoding(kargs['maximum_position_encoding'], self.d_model)

        self.dec_layers = [DecoderLayer(**kargs)
                           for _ in range(self.num_layers)]
        self.dropout = tf.keras.layers.Dropout(kargs['rate'])

    def call(self, x, enc_output, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))  # 스케일링 기울기가 너무크거가 작아지는것을 방지 모델안정적으로
        x += self.pos_encoding[:, :seq_len, :] # 순서정보 유

        x = self.dropout(x)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

In [ ]:
# 트랜스포머 모델
STD_INDEX = 1

class Transformer(tf.keras.Model):
  def __init__(self, **kargs):
      super(Transformer, self).__init__()

      self.end_token_idx = kargs['end_token_idx']

      self.encoder = Encoder(**kargs)
      self.decoder = Decoder(**kargs)

      self.final_layer = tf.keras.layers.Dense(kargs['target_vocab_size'])

  def __call__(self, x):
      inp, tar = x

      enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
      enc_output = self.encoder(inp, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

      # dec_output.shape == (batch_size, tar_seq_len, d_model)
      dec_output, _ = self.decoder(
          tar, enc_output, look_ahead_mask, dec_padding_mask)

      final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

      return final_output

  def inference(self, x):
      inp = x
      tar = tf.expand_dims([STD_INDEX], 0)

      enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
      enc_output = self.encoder(inp, enc_padding_mask)

      predict_tokens = list()
      for t in range(0, MAX_SEQUENCE):
          dec_output, _ = self.decoder(tar, enc_output, look_ahead_mask, dec_padding_mask)
          final_output = self.final_layer(dec_output)
          outputs = tf.argmax(final_output, -1).numpy()
          pred_token = outputs[0][-1]
          if pred_token == self.end_token_idx:
              break
          predict_tokens.append(pred_token)
          tar = tf.expand_dims([STD_INDEX] + predict_tokens, 0)
          _, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)

      return predict_tokens

# 모델

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)


In [ ]:
model = Transformer(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=loss,
              metrics=[accuracy])

In [ ]:
# 콜벡
# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=10)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)

checkpoint_path = model_name + '.weights.h5'
checkpoint_dir = './'

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:
# 학습
history = model.fit([index_inputs,index_outputs], index_targets, batch_size = BATCH_SIZE,epochs=EPOCHS,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])